<a href="https://colab.research.google.com/github/cray0101/DevvitApps/blob/main/distributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# Default H-1B applicant estimates (relative scale)
countries = {
    'India': 100000,
    'China': 18000,
    'Canada': 6000,
    'Philippines': 6000,
    'Germany': 5000,
    'UK': 5000,
    'Brazil': 4000,
    'Mexico': 3500,
    'Nigeria': 3000,
    'Estonia': 500
}

total_visas = 65000

def compute_allocations(countries_dict, alpha, cap_percent=None):
    applicants = pd.Series(countries_dict)
    if cap_percent:
        cap = total_visas * cap_percent
        capped = applicants.copy()
        capped[capped > cap] = cap
        scaled = capped / capped.sum() * total_visas
    else:
        weights = applicants ** alpha
        scaled = weights / weights.sum() * total_visas
    return scaled

def update(alpha=0.33, cap=False, **app_counts):
    # Gather dynamic applicant inputs
    dynamic_countries = {c: app_counts[c] for c in countries}
    if cap:
        allocations = compute_allocations(dynamic_countries, alpha=1.0, cap_percent=0.07)
        method = "7% Cap"
    else:
        allocations = compute_allocations(dynamic_countries, alpha=alpha)
        method = f"Dampening (alpha={alpha:.2f})"

    df = pd.DataFrame({
        'Country': allocations.index,
        'Applicants': [dynamic_countries[c] for c in allocations.index],
        'Allocated Visas': allocations.values
    })
    df['Odds per Applicant (%)'] = (df['Allocated Visas'] / df['Applicants']) * 100
    df.sort_values('Allocated Visas', ascending=False, inplace=True)

    plt.figure(figsize=(12, 6))
    plt.bar(df['Country'], df['Allocated Visas'], color='skyblue')
    plt.title(f'H-1B Visa Allocation — {method}')
    plt.ylabel('Allocated Visas')
    plt.xlabel('Country')
    plt.xticks(rotation=45)
    plt.ylim(0, total_visas * 0.8)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    display(df)

# Create UI controls
alpha_slider = widgets.FloatSlider(value=0.33, min=0.05, max=1.0, step=0.01, description='Alpha:')
cap_checkbox = widgets.Checkbox(value=False, description='Apply 7% Cap')

# Create input fields for applicant counts
app_sliders = [
    widgets.IntText(value=cnt, description=c, layout=widgets.Layout(width='200px'))
    for c, cnt in countries.items()
]

ui = widgets.VBox([
    alpha_slider,
    cap_checkbox,
    widgets.Label("Adjust Applicant Counts:"),
    widgets.GridBox(app_sliders, layout=widgets.Layout(grid_template_columns="repeat(2, 220px)"))
])

out = widgets.interactive_output(update, {
    'alpha': alpha_slider,
    'cap': cap_checkbox,
    **{c: slider for c, slider in zip(countries.keys(), app_sliders)}
})

display(ui, out)

Output()